# Import Repo of Sepsis Simulator

In [ ]:
!git clone https://github.com/clinicalml/gumbel-max-scm.git

Cloning into 'gumbel-max-scm'...
remote: Enumerating objects: 113, done.
remote: Counting objects: 100% (3/3), done.
remote: Compressing objects: 100% (3/3), done.
remote: Total 113 (delta 0), reused 0 (delta 0), pack-reused 110
Receiving objects: 100% (113/113), 1.48 MiB | 4.01 MiB/s, done.
Resolving deltas: 100% (28/28), done.


In [ ]:
#Enable importing code from parent directory
import os, sys
simulator_path = os.path.abspath('./gumbel-max-scm')
sys.path.insert(1, simulator_path)

In [ ]:
!pip install pymdptoolbox

  Preparing metadata (setup.py) ... done
  Created wheel for pymdptoolbox: filename=pymdptoolbox-4.0b3-py3-none-any.whl size=25657 sha256=bb44fa85cd8dee155a5cd718136824544d1fd5e71045a48790cb4a6c22766117
  Stored in directory: /root/.cache/pip/wheels/2b/e7/c7/d7abf9e309f3573a934fed2750c70bd75d9e9d901f7f16e183
Successfully built pymdptoolbox


**IMPORTANT NOTE:** At this stage, to reproduce our experiments, one must modify line 38 of `gumbel-max-scm/sepsisSimDiabetes/DataGenerator.py` so that it reads:

```
mdp = MDP(init_state_idx=%state%,
          policy_array=policy, policy_idx_type=policy_idx_type,
          p_diabetes=p_diabetes)

```

We have essentially set the initial state to a fixed value so that we may estimate the Q-function from that state. Additionally, line 58 of the same file must be modified to:

```
mdp.state = mdp.get_new_state(state_idx = %state%)
```

In [ ]:
import numpy as np
import cf.counterfactual as cf
import cf.utils as utils
import pandas as pd
import pickle
import itertools as it
from tqdm import tqdm_notebook as tqdm
from scipy.linalg import block_diag

# Sepsis Simulator code
from sepsisSimDiabetes.State import State
from sepsisSimDiabetes.Action import Action
from sepsisSimDiabetes.DataGenerator import DataGenerator
import sepsisSimDiabetes.MDP as simulator

import mdptoolboxSrc.mdp as mdptools

import matplotlib.pyplot as plt
import seaborn as sns
import warnings
warnings.simplefilter(action='ignore', category=FutureWarning)

import numpy as np
# For optimization
from scipy.optimize import Bounds, BFGS
from scipy.optimize import LinearConstraint, NonlinearConstraint, minimize
# For generating dataset
import sklearn.datasets as dt

ZERO = 1e-7

# Set up Variables and Functions

Code taken from [Oberst and Sontag](https://github.com/clinicalml/gumbel-max-scm/blob/master/plots-main-paper.ipynb).

Set up important variables

In [ ]:
SEED = 1
np.random.seed(SEED)
NSIMSAMPS = 100000  # Samples to draw from the simulator
NSTEPS = 20  # Max length of each trajectory
NCFSAMPS = 5  # Counterfactual Samples per observed sample
DISCOUNT_Pol = 0.99 # Used for computing optimal policies
DISCOUNT = 1 # Used for computing actual reward
PHYS_EPSILON = 0.05 # Used for sampling using physician pol as eps greedy

# Option 1: Use bootstrapping w/replacement on the original NSIMSAMPS to estimate errors
USE_BOOSTRAP=True
N_BOOTSTRAP = 100

# Option 2: Use repeated sampling (i.e., NSIMSAMPS fresh simulations each time) to get error bars;
# This is done in the appendix of the paper, but not in the main paper
N_REPEAT_SAMPLING = 1

# These are properties of the simulator, do not change
n_actions = Action.NUM_ACTIONS_TOTAL
n_components = 2

# These are added as absorbing states
n_states_abs = State.NUM_OBS_STATES + 2
discStateIdx = n_states_abs - 1
deadStateIdx = n_states_abs - 2

# Number of runs for calculating MSE
RUNS = 20
# Number of episodes over which we average an OPE estimate
N = 1000

In [ ]:
hr_state_mapping = ['Low', 'Normal', 'High']
sbp_state_mapping = ['Low', 'Normal', 'High']
o2_state_mapping = ['Low', 'Normal']
glu_state_mapping = ['Very Low', 'Low', 'Normal', 'High', 'Very High']
abx_state_mapping = ['Off', 'On']
vaso_state_mapping = ['Off', 'On']
vent_state_mapping = ['Off', 'On']
diab_state_mapping = ['No', 'Yes']

Set up base for behaviour and evaluation policies

In [ ]:
import zipfile
with zipfile.ZipFile("gumbel-max-scm/data/diab_txr_mats-replication.zip", 'r') as zip_ref:
    zip_ref.extractall("gumbel-max-scm/data")

In [ ]:
# Get the transition and reward matrix from file
with open("gumbel-max-scm/data/diab_txr_mats-replication.pkl", "rb") as f:
    mdict = pickle.load(f)

tx_mat = mdict["tx_mat"]
r_mat = mdict["r_mat"]

In [ ]:
from scipy.linalg import block_diag

tx_mat_full = np.zeros((n_actions, State.NUM_FULL_STATES, State.NUM_FULL_STATES))
r_mat_full = np.zeros((n_actions, State.NUM_FULL_STATES, State.NUM_FULL_STATES))

# Easily accessible variables
A = n_actions
S = State.NUM_FULL_STATES

for a in range(n_actions):
    tx_mat_full[a, ...] = block_diag(tx_mat[0, a, ...], tx_mat[1, a,...])
    r_mat_full[a, ...] = block_diag(r_mat[0, a, ...], r_mat[1, a, ...])

In [ ]:
fullMDP = cf.MatrixMDP(tx_mat_full, r_mat_full)
fullPol = fullMDP.policyIteration(discount=DISCOUNT_Pol, eval_type=1)

#The behavior policy is the fully random policy
randPol = np.ones(fullPol.shape)/(fullPol.shape[1])

In [ ]:
#We want the expected reward of starting in a state and taking an action
R = np.swapaxes(np.mean(r_mat_full, axis=-1), 0, 1)
R.shape

In [ ]:
#To handle -1 states and -1 actions
def pad_policy(policy, val=1):
  #Add a column of zeroes to the end
  policy = np.concatenate((policy, np.full((policy.shape[0], 1), val)), axis=1)
  #Add a row of zeroes at the end
  policy = np.concatenate((policy, np.full((1, policy.shape[1]), val)), axis=0)
  return policy

# Load repo

In [ ]:
!git clone https://github.com/ai4ai-lab/Factored-Action-Spaces-for-OPE.git

In [ ]:
#Enable importing code from parent directory
import os, sys
main_folder = os.path.abspath('./Factored-Action-Spaces-for-OPE')
sys.path.insert(1, main_folder)

# From Patient State 136, With Diabetes

In [ ]:
#The patient has diabetes
PROB_DIAB = 1.0

### State Analysis

In [ ]:
#Instantiate a state based on the idx and get the state vector
testState = State(state_idx = 136, diabetic_idx=1)
vec = testState.get_state_vector()

print(vec)

print(f'Heart Rate: {hr_state_mapping[vec[0]]}')
print(f'Systolic Blood Pressure: {sbp_state_mapping[vec[1]]}')
print(f'Percent Oxygen: {o2_state_mapping[vec[2]]}')
print(f'Glucose Level: {glu_state_mapping[vec[3]]}')
print(f'Antibiotics: {abx_state_mapping[vec[4]]}')
print(f'Vasopressors: {vaso_state_mapping[vec[5]]}')
print(f'Ventilator: {vent_state_mapping[vec[6]]}')
print(f'Diabetes: {testState.diabetic_idx}')

### Generate Data From Behaviour Policy

Run the data generator

In [ ]:
dgen = DataGenerator()
states, actions, lengths, rewards, diab, emp_tx_totals, emp_r_totals = dgen.simulate(
    NSIMSAMPS, NSTEPS, policy=randPol, policy_idx_type='full',
    p_diabetes=PROB_DIAB, use_tqdm=False) #True, tqdm_desc='Behaviour Policy Simulation')

obs_samps = utils.format_dgen_samps(
    states, actions, rewards, diab, NSTEPS, NSIMSAMPS)

Convert data into array format

In [ ]:
time = np.arange(NSTEPS)
times = np.stack(axis=0, arrays=[time]*NSIMSAMPS)
times = times[..., np.newaxis]

nf_tr_b = np.concatenate((times, states[:, 0:NSTEPS, :], actions, rewards, states[:, 1:, :]), axis=2)
nf_tr_b.shape

(100000, 20, 5)

In [ ]:
print(nf_tr_b)

[[[  0. 136.   0.  -1. 168.]
  [  1. 168.  -1.   0.  -1.]
  [  2.  -1.  -1.   0.  -1.]
  ...
  [ 17.  -1.  -1.   0.  -1.]
  [ 18.  -1.  -1.   0.  -1.]
  [ 19.  -1.  -1.   0.  -1.]]

 [[  0. 136.   3.  -1. 227.]
  [  1. 227.  -1.   0.  -1.]
  [  2.  -1.  -1.   0.  -1.]
  ...
  [ 17.  -1.  -1.   0.  -1.]
  [ 18.  -1.  -1.   0.  -1.]
  [ 19.  -1.  -1.   0.  -1.]]

 [[  0. 136.   7.   0. 223.]
  [  1. 223.   3.   0. 219.]
  [  2. 219.   1.   0. 218.]
  ...
  [ 17.  -1.  -1.   0.  -1.]
  [ 18.  -1.  -1.   0.  -1.]
  [ 19.  -1.  -1.   0.  -1.]]

 ...

 [[  0. 136.   2.   0. 377.]
  [  1. 377.   4.   0. 372.]
  [  2. 372.   7.   0. 463.]
  ...
  [ 17.  -1.  -1.   0.  -1.]
  [ 18.  -1.  -1.   0.  -1.]
  [ 19.  -1.  -1.   0.  -1.]]

 [[  0. 136.   5.   0. 222.]
  [  1. 222.   1.   0. 218.]
  [  2. 218.   7.  -1. 231.]
  ...
  [ 17.  -1.  -1.   0.  -1.]
  [ 18.  -1.  -1.   0.  -1.]
  [ 19.  -1.  -1.   0.  -1.]]

 [[  0. 136.   7.  -1. 231.]
  [  1. 231.  -1.   0.  -1.]
  [  2.  -1.  -1.   0.  -1

In [ ]:
randPol = pad_policy(randPol)

### Varying Episodes $\epsilon_{e} = 0.4$ (Policy Divergence $4.8^{20}$)

Set up evaluation policy, generate data and convert into factored format

In [ ]:
EVAL_EPSILON = 0.4

evalPolSoft = np.copy(fullPol)
evalPolSoft[evalPolSoft == 1] = 1 - EVAL_EPSILON
evalPolSoft[evalPolSoft == 0] = EVAL_EPSILON / (n_actions - 1)

In [ ]:
# Calculate policy divergence from Voloshin et al.
D = 0
for state in range(randPol.shape[0] - 1):
    for action in range(randPol.shape[1] - 1):
        difference = evalPolSoft[state, action]/randPol[state, action]
        D = max(D, difference)
print(D)
shorter_D = round(D,2)

4.8


In [ ]:
dgen = DataGenerator()
states, actions, lengths, rewards, diab, emp_tx_totals, emp_r_totals = dgen.simulate(
    NSIMSAMPS, NSTEPS, policy=evalPolSoft, policy_idx_type='full',
    p_diabetes=PROB_DIAB, use_tqdm=False) #True, tqdm_desc='Behaviour Policy Simulation')

obs_samps = utils.format_dgen_samps(
    states, actions, rewards, diab, NSTEPS, NSIMSAMPS)

In [ ]:
time = np.arange(NSTEPS)
times = np.stack(axis=0, arrays=[time]*NSIMSAMPS)
times = times[..., np.newaxis]

nf_tr_e = np.concatenate((times, states[:, 0:NSTEPS, :], actions, rewards, states[:, 1:, :]), axis=2)
nf_tr_e.shape

(100000, 20, 5)

In [ ]:
print(nf_tr_e)

[[[  0. 136.   0.   0. 136.]
  [  1. 136.   2.   0.  57.]
  [  2.  57.   3.  -1. 227.]
  ...
  [ 17.  -1.  -1.   0.  -1.]
  [ 18.  -1.  -1.   0.  -1.]
  [ 19.  -1.  -1.   0.  -1.]]

 [[  0. 136.   2.   0.  57.]
  [  1.  57.   4.  -1.  68.]
  [  2.  68.  -1.   0.  -1.]
  ...
  [ 17.  -1.  -1.   0.  -1.]
  [ 18.  -1.  -1.   0.  -1.]
  [ 19.  -1.  -1.   0.  -1.]]

 [[  0. 136.   2.   0. 449.]
  [  1. 449.   5.   0. 462.]
  [  2. 462.   7.   0. 471.]
  ...
  [ 17.  -1.  -1.   0.  -1.]
  [ 18.  -1.  -1.   0.  -1.]
  [ 19.  -1.  -1.   0.  -1.]]

 ...

 [[  0. 136.   2.   0. 145.]
  [  1. 145.   3.   0. 147.]
  [  2. 147.   2.   0.  57.]
  ...
  [ 17.  -1.  -1.   0.  -1.]
  [ 18.  -1.  -1.   0.  -1.]
  [ 19.  -1.  -1.   0.  -1.]]

 [[  0. 136.   7.   0. 223.]
  [  1. 223.   6.   0. 221.]
  [  2. 221.   4.   0. 132.]
  ...
  [ 17.  -1.  -1.   0.  -1.]
  [ 18.  -1.  -1.   0.  -1.]
  [ 19.  -1.  -1.   0.  -1.]]

 [[  0. 136.   2.   0.  57.]
  [  1.  57.   3.   0. 147.]
  [  2. 147.   2.   0. 145

In [ ]:
evalPolSoft = pad_policy(evalPolSoft)

In [ ]:
unique_states = np.unique(nf_tr_b[:, :, 1])
unique_states = unique_states[unique_states != -1].astype(int)
Sunq = len(unique_states)

In [ ]:
unique_actions = np.unique(nf_tr_b[:, :, 2])
unique_actions = unique_actions[unique_actions != -1].astype(int)
Aunq = len(unique_actions)

In [ ]:
import policy_estimators as pe
from sklearn.metrics import mean_squared_error
import gc

# Objective function
#alpha has shape (SxAXDx3)
def OPE_MSE(alpha, discount_factor, D):
    boundary = D*Aunq*Sunq
    pi_b_d = alpha[0:boundary]
    pi_e_d = alpha[boundary:2*boundary]
    r_d = alpha[2*boundary:]
    print(pi_b_d)
    print(pi_e_d)
    print(r_d)
    #Reshape policies and reward
    factored_pi_b = np.reshape(pi_b_d, newshape=(D, Sunq, Aunq))
    factored_pi_e = np.reshape(pi_e_d, newshape=(D, Sunq, Aunq))
    factored_r = np.reshape(r_d, newshape=(D,Sunq,Aunq))
    #Expand actions to full
    expanded1_pi_b = np.zeros((D,Sunq,A))
    expanded1_pi_e = np.zeros((D,Sunq,A))
    expanded1_r = np.zeros((D,Sunq,A))
    expanded1_pi_b[:, :, unique_actions] = factored_pi_b
    expanded1_pi_e[:, :, unique_actions] = factored_pi_e
    expanded1_r[:, :, unique_actions] = factored_r
    #Expand states to full
    expanded2_pi_b = np.zeros((D,S,A))
    expanded2_pi_e = np.zeros((D,S,A))
    factored_r = np.zeros((D,S,A))
    expanded2_pi_b[:, unique_states, :] = expanded1_pi_b
    expanded2_pi_e[:, unique_states, :] = expanded1_pi_e
    factored_r[:, unique_states, :] = expanded1_r

    #Pad each factored policy
    action_spaces = [i for i in range(D)]
    factored_pi_b = []
    factored_pi_e = []
    for d in action_spaces:
      factored_pi_b.append(pad_policy(expanded2_pi_b[d, :, :]))
      factored_pi_e.append(pad_policy(expanded2_pi_e[d, :, :]))
    gc.collect()

    estimates = np.zeros(RUNS)

    #Obtain OPE estimates from each run
    for run in range(RUNS):
      transition_data = nf_tr_b[run*N:(run+1)*N, :, :].astype(int)
      factored_transition_data = np.zeros((N, NSTEPS, D, 5))
      #Generate factored transition data based on transition data and variables
      global rhos
      for d in range(D):
        factored_transition_data[:, :, d, :] = transition_data[:, :, :]
        factored_transition_data[:, :, d, 3] = factored_r[d, transition_data[:, :, 1], transition_data[:, :, 2]]

        #Transition probabilities
        s_list = factored_transition_data[:,:,d,1].astype(int)
        a_list = factored_transition_data[:,:,d,2].astype(int)
        p_b = factored_pi_b[d][s_list, a_list]
        p_e = factored_pi_e[d][s_list, a_list]

        # Per-trajectory cumulative importance ratios, take the product
        rhos[run, d] = np.mean((p_e / p_b).prod(-1))

      #Obtain OPE estimate
      OPE_estimate, _ = pe.off_policy_DecIS_estimator(factored_transition_data,
                                                discount_factor,
                                                action_spaces,
                                                factored_pi_b,
                                                factored_pi_e)
      estimates[run] = OPE_estimate
      gc.collect()

    #Obtain on policy estimate
    on_policy_estimate = pe.on_policy_Q_estimate(nf_tr_e, discount_factor)
    #Calculate MSE
    MSE = mean_squared_error(estimates, [on_policy_estimate]*RUNS)

    global avg_estimate, on_policy_val
    avg_estimate = np.mean(estimates)
    on_policy_val = on_policy_estimate

    print(MSE)
    print('--------------------')
    return MSE/10.0

In [ ]:
def jac(alpha, discount_factor, D):

  boundary = D*Aunq*Sunq
  gradvals = np.zeros(3*boundary)

  #Calculate bias
  global avg_estimate, on_policy_val
  bias = avg_estimate - on_policy_val

  gradvals[:boundary] = 2*bias*avg_estimate/N
  gradvals[:boundary] = gradvals[:boundary]/alpha[:boundary]

  gradvals[boundary:2*boundary] = -2*bias*avg_estimate/N
  gradvals[boundary:2*boundary] = gradvals[boundary:2*boundary]/alpha[boundary:2*boundary]
  global rhos
  avg_rhos = np.mean(rhos, axis=0)
  mini = Aunq*Sunq
  for d in range(D):
    gradvals[2*boundary + d*mini: 2*boundary + (d+1)*mini] = (2*bias*discount_factor/N)*avg_rhos[d]
  gc.collect()
  print(gradvals)
  print(gradvals.shape)
  return gradvals

In [ ]:
def hess(alpha, discount_factor, D):
  boundary = D*Aunq*Sunq
  pi_b_d = alpha[0:boundary]
  pi_e_d = alpha[boundary:2*boundary]
  r_d = alpha[2*boundary:]
  print(pi_b_d)
  print(pi_e_d)
  print(r_d)
  #Reshape policies and reward
  factored_pi_b = np.reshape(pi_b_d, newshape=(D, Sunq, Aunq))
  factored_pi_e = np.reshape(pi_e_d, newshape=(D, Sunq, Aunq))
  factored_r = np.reshape(r_d, newshape=(D,Sunq,Aunq))
  #Expand actions to full
  expanded1_pi_b = np.zeros((D,Sunq,A))
  expanded1_pi_e = np.zeros((D,Sunq,A))
  expanded1_r = np.zeros((D,Sunq,A))
  expanded1_pi_b[:, :, unique_actions] = factored_pi_b
  expanded1_pi_e[:, :, unique_actions] = factored_pi_e
  expanded1_r[:, :, unique_actions] = factored_r
  #Expand states to full
  expanded2_pi_b = np.zeros((D,S,A))
  expanded2_pi_e = np.zeros((D,S,A))
  factored_r = np.zeros((D,S,A))
  expanded2_pi_b[:, unique_states, :] = expanded1_pi_b
  expanded2_pi_e[:, unique_states, :] = expanded1_pi_e
  factored_r[:, unique_states, :] = expanded1_r

  #Pad each factored policy
  action_spaces = [i for i in range(D)]
  factored_pi_b = []
  factored_pi_e = []
  for d in action_spaces:
    factored_pi_b.append(pad_policy(expanded2_pi_b[d, :, :]))
    factored_pi_e.append(pad_policy(expanded2_pi_e[d, :, :]))
  gc.collect()

  estimates = np.zeros(RUNS)

  #Obtain OPE estimates from each run
  for run in range(RUNS):
    transition_data = nf_tr_b[run*N:(run+1)*N, :, :].astype(int)
    decomposed_Qs = np.zeros(len(action_spaces))
    for d in range(D):
      t_list = transition_data[:, :, 0].astype(int)
      s_list = transition_data[:, :, 1].astype(int)
      a_list = transition_data[:, :, 2].astype(int)
      r_list = factored_r[d, s_list, a_list]

      # Per-trajectory returns (discounted cumulative rewards)
      G = (r_list * np.power(discount_factor, t_list)).sum(axis=-1)

      #Transition probabilities
      p_b = factored_pi_b[d][s_list, a_list]
      p_e = factored_pi_e[d][s_list, a_list]

      # Per-trajectory cumulative importance ratios, take the product
      rho = (p_e / p_b).prod(-1)
      prod = G*rho[-1]
      #Weight with IS ratio then average
      decomposed_Qs[d] = np.average(prod)
      gc.collect()
    #Sum up decomposed Q estimates
    estimates[run] = np.sum(decomposed_Qs)

  #Obtain on policy estimate
  on_policy_estimate = pe.on_policy_Q_estimate(nf_tr_e, discount_factor)
  #Calculate MSE
  bias = np.mean(estimates) - on_policy_estimate

  gradvals = np.full(3*boundary, 2*bias)
  gradvals[boundary:2*boundary] = -2*bias

In [ ]:
def optimize_alpha(discount_factor, D, max_R, factor_pi_b=False, factor_pi_e=False):
    np.random.seed(1)
    boundary = D*Aunq*Sunq
    # Initialize alphas to random values
    alpha_0 = np.zeros(boundary*3)
    alpha_0[:boundary//2] = (randPol[unique_states, :])[:, unique_actions].flatten()
    alpha_0[boundary//2:boundary] = (randPol[unique_states, :])[:, unique_actions].flatten()
    alpha_0[boundary:3*boundary//2] = (evalPolSoft[unique_states, :])[:, unique_actions].flatten()
    alpha_0[3*boundary//2:2*boundary] = (evalPolSoft[unique_states, :])[:, unique_actions].flatten()
    alpha_0[2*boundary:5*boundary//2] = (R[unique_states, :])[:, unique_actions].flatten()/D
    alpha_0[5*boundary//2:] = (R[unique_states, :])[:, unique_actions].flatten()/D
    #Set bounds on policy and reward
    lower_bound = np.zeros(boundary*3)
    lower_bound[:2*boundary] = alpha_0[:2*boundary]
    lower_bound[2*boundary:] = -max_R
    upper_bound = np.full(boundary*3, 1)
    upper_bound[2*boundary:] = max_R
    # Define the bounds
    bounds_alpha = Bounds(lower_bound, upper_bound)
    gc.collect()
    print(f'Checkpoint 1 {3*boundary}')
    #Set linear and non-linear constraints on policy and reward
    constraints = []
    #Policies must add up to 1
    constraint_base = np.zeros(D*Sunq*Aunq*3)
    for d in range(D):
      for s in range(Sunq):
        start_index = d*Sunq*Aunq + s*Aunq
        constraint_base[:] = 0.0
        constraint_base[start_index:start_index + Aunq] = 1
        constraints.append(LinearConstraint(constraint_base, [1], [1]))
        gc.collect()
        constraint_base[:] = 0.0
        constraint_base[boundary + start_index:boundary + start_index + Aunq] = 1
        constraints.append(LinearConstraint(constraint_base, [1], [1]))
        gc.collect()
    print('Checkpoint 2')
    #Factored rewards must add to full reward
    for s in range(Sunq):
      for a in range(Aunq):
        constraint_base[:] = 0.0
        index_list = np.arange(D)*Sunq*Aunq + (s*Aunq + a + 2*boundary)
        constraint_base[index_list] = 1
        constraints.append(LinearConstraint(constraint_base, [R[s,a]], [R[s,a]]))
        gc.collect()
        #If we know that the behaviour policy can be factored
        if factor_pi_b:
          constraint_base[:] = 0.0
          index_list = np.arange(D)*Sunq*Aunq + (s*Aunq + a)
          constraint_base[index_list] = 1
          def prodf(x):
            masked_arr = x*constraint_base
            return np.prod(masked_arr, where=(masked_arr!=0))
          constraints.append(NonlinearConstraint(prodf, [randPol[s,a]], [randPol[s,a]]))
        #If we know that the evaluation policy can be factored
        if factor_pi_e:
          constraint_base[:] = 0.0
          index_list = np.arange(D)*Sunq*Aunq + (s*Aunq + a + 1*boundary)
          constraint_base[index_list] = 1
          def prodf(x):
            masked_arr = x*constraint_base
            return np.prod(masked_arr, where=(masked_arr!=0))
          constraints.append(NonlinearConstraint(prodf, [evalPolSoft[s,a]], [evalPolSoft[s,a]]))

    print('Checkpoint 3')
    # Find the optimal value of alpha
    result = minimize(OPE_MSE, alpha_0, args = (discount_factor, D), method='SLSQP', jac=jac,
                      #hess=BFGS(),
                      constraints=constraints,
                      bounds=bounds_alpha)
    # The optimized value of alpha lies in result.x
    alpha = result.x
    return alpha

In [ ]:
max_R = np.max(r_mat_full)
D = 2
#Define global variables
rhos = np.zeros((RUNS, D))
avg_estimate = 0
on_policy_val = 0
factorisation = optimize_alpha(DISCOUNT_Pol, D, max_R, factor_pi_b=False, factor_pi_e=False)
boundary = D*Sunq*Aunq
print(factorisation[0:boundary])
print(factorisation[boundary:2*boundary])
print(factorisation[2*boundary:])

Checkpoint 1 24048
Checkpoint 2
Checkpoint 3
[0.125 0.125 0.125 ... 0.125 0.125 0.125]
[0.05714286 0.05714286 0.05714286 ... 0.05714286 0.6        0.05714286]
[-0.14409722 -0.14409722 -0.14409722 ... -0.14409722 -0.14409722
 -0.14409722]
3.5084325843666107
--------------------
[ 0.01048075  0.01048075  0.01048075 ... -0.00100111 -0.00100111
 -0.00100111]
(24048,)


In [ ]:
max_R = np.max(r_mat_full)
D = 3
factorisation = optimize_alpha(DISCOUNT_Pol, D, max_R, factor_pi_b=False, factor_pi_e=False)
boundary = D*S*A
print(factorisation[0:boundary])
print(factorisation[boundary:2*boundary])
print(factorisation[2*boundary:])